In [1]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

2023-12-19 15:38:46.653744: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


'2.9.1'

## Load Model With Keras

In [3]:
model = keras.models.load_model('./saved_model/xception_bean_leaf_16_0.947.h5')

2023-12-19 15:42:04.980692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-19 15:42:04.988803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-19 15:42:04.989446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-19 15:42:04.990351: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [12]:
img = load_img('./data/val/healthy/healthy_val.1.jpg',target_size=(299,299))

In [13]:
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [14]:
preds = model.predict(X)

1/1 [==============================] - 0s 60ms/step


In [15]:
preds

array([[-6.301956 , -2.1655529,  8.311718 ]], dtype=float32)

In [16]:
classes = [
    'angular leaf spot',
    'bean rust',
    'healthy',
]

In [17]:
dict(zip(classes,preds[0]))

{'angular leaf spot': -6.301956, 'bean rust': -2.1655529, 'healthy': 8.311718}

## Convert Keras to TF-Lite

Here we want to convert keras model to tf-lite, in order tor reduce the model size. Smaller model will lead small docker size as well

In [22]:
# convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the model
with open('./saved_model/bean-leaf-model.tflite','wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpcfx5uc8b/assets


INFO:tensorflow:Assets written to: /tmp/tmpcfx5uc8b/assets
2023-12-19 16:04:36.776046: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-19 16:04:36.776109: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-12-19 16:04:36.776286: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpcfx5uc8b
2023-12-19 16:04:36.805939: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-12-19 16:04:36.805981: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpcfx5uc8b
2023-12-19 16:04:36.934779: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-12-19 16:04:37.526572: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpcfx5uc8b
2023-12-19 16:04:37.770167: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

In [25]:
!ls -lh ./saved_model

total 243M
-rw-r--r-- 1 jovyan jovyan 81M Dec 19 16:04 bean-leaf-model.tflite
-rw-rw-r-- 1 jovyan jovyan 83M Dec 17 16:16 xception_bean_leaf_16_0.947.h5
-rw-rw-r-- 1 jovyan jovyan 80M Dec 16 17:17 xception_v1_03_0.925.h5


successfully reduce model size by 2 MB. The reduction in size is not that significant, possibly because the initial size of the model is not that large

#### Use converted tflite model

In [26]:
import tensorflow.lite as tflite

In [27]:
# interprete converted model
interpreter = tflite.Interpreter(model_path='./saved_model/bean-leaf-model.tflite')
interpreter.allocate_tensors()

# get input index
input_index = interpreter.get_input_details()[0]['index']
# get output index
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [30]:
# initialing  input manually based on the corresponding index
interpreter.set_tensor(input_index,X)
interpreter.invoke()

# preds the test image with converted tflite model
preds = interpreter.get_tensor(output_index)

In [31]:
classes = [
    'angular leaf spot',
    'bean rust',
    'healthy',
]

dict(zip(classes,preds[0]))

{'angular leaf spot': -6.3019586, 'bean rust': -2.1655617, 'healthy': 8.311728}

## Removing TF dependency

We still use tensorflow dependency for preprocessing image. So here we want to completely remove tensorflow dependency for avoid using big library dependency only for doing simple task

Here we will leverage lightweight preprocessing library from Alexey

In [33]:
!pip install keras_image_helper

In [49]:
!pip install tflite_runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.55.2 requires numpy<1.23,>=1.18, but you have numpy 1.26.2 which is incompatible.


In [1]:
from keras_image_helper import create_preprocessor
import tflite_runtime.interpreter as tflite

In [2]:
interpreter = tflite.Interpreter(model_path='./saved_model/bean-leaf-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [4]:
url = 'https://drive.google.com/uc?export=view&id=1MGvOaIy94muwFCofOd88pNRszUUiwdvf'
X = preprocessor.from_url(url)

In [5]:
interpreter.set_tensor(input_index,X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [7]:
classes = [
    'angular leaf spot',
    'bean rust',
    'healthy',
]
dict(zip(classes,preds[0]))

{'angular leaf spot': 9.285762, 'bean rust': -2.1980135, 'healthy': -8.247787}